<a href="https://colab.research.google.com/github/amankiitg/LLM_Prod/blob/main/VLLM_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers accelerate sentencepiece torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.4 MB/s eta 0:00:00


In [ ]:
!pip install -q vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.

In [ ]:
import platform, torch, vllm
print("vLLM:", vllm.__version__)
print("PyTorch:", torch.__version__)
print("CUDA available?", torch.cuda.is_available())

In [ ]:
!pip install -q matplotlib
import time, itertools, os, json, math, random
from pathlib import Path

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from vllm import LLM, SamplingParams

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# --- Setup ---
MODEL_NAME = "Qwen/Qwen1.5-1.8B"
tokenizer_hf = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

if tokenizer_hf.pad_token is None:
    tokenizer_hf.pad_token = tokenizer_hf.eos_token

tokenizer_hf.padding_side = 'left'

model_hf = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

pipe_hf = pipeline(
    "text-generation",
    model=model_hf,
    tokenizer=tokenizer_hf,
    max_new_tokens=10,
    batch_size=4,
    do_sample=False,
)

# --- Prompt ---
SYSTEM_PROMPT = """Task: Determine if code needs migration from Python to C++.
Rules:
- If the code is Python: answer "Yes"
- If the code is C++ or other language: answer "No"
- Answer with only one word: "Yes" or "No"

Examples:
Code: def func(): pass
Answer: Yes

Code: #include <iostream>
Answer: No
"""

# --- Classification Function ---
def classify_code(snippets):
    prompts = [
        f"""{SYSTEM_PROMPT}

Code: {code}
Answer:"""
        for code in snippets
    ]

    outputs = pipe_hf(prompts, return_full_text=False)

    def extract_answer(text):
        response = text[0]["generated_text"].strip()
        # Take first line and first word
        first_word = response.split('\n')[0].split()[0] if response.split() else ""

        if "yes" in first_word.lower():
            return "Yes"
        elif "no" in first_word.lower():
            return "No"
        return f"Unclear: {first_word}"

    return [extract_answer(out) for out in outputs]

# --- Test Snippets ---
python_snippet = """def add(a, b):
    return a + b
if __name__ == "__main__":
    print(add(2, 3))"""

cpp_snippet = """#include <iostream>
int add(int a, int b) {
    return a + b;
}
int main() {
    std::cout << add(2, 3) << std::endl;
    return 0;
}"""

javascript_snippet = """function add(a, b) {
    return a + b;
}
console.log(add(2, 3));"""

# --- Run Classification ---
snippets = [python_snippet, cpp_snippet, javascript_snippet]
labels = classify_code(snippets)

for i, (code, label) in enumerate(zip(snippets, labels), 1):
    print(f"--- Snippet {i} ---")
    lang = "Python" if "def " in code else "C++" if "#include" in code else "JavaScript"
    print(f"Language: {lang}")
    print(f"First few lines: {code.split(chr(10))[0]}")
    print(f"→ Needs migration to C++: {label}")
    print()

In [ ]:
import gc, torch

del pipe_hf, model_hf, tokenizer_hf      # remove the HF objects
gc.collect()                             # reclaim Python refs
torch.cuda.empty_cache()                 # actually free GPU memory


In [ ]:
from vllm import LLM, SamplingParams
import time

# --- Model Configuration ---
MODEL_NAME = "Qwen/Qwen1.5-1.8B"

# --- Initialize vLLM model with fixed configuration ---
llm_vllm = LLM(
    model=MODEL_NAME,
    trust_remote_code=True,
    dtype="half",
    gpu_memory_utilization=0.85,  # Increased from 0.2
    max_model_len=1024,          # Reduced from default 32768

)

# --- Sampling parameters ---
sampling = SamplingParams(
    max_tokens=10,
    temperature=0.0,
    stop=["\n", "Code:", "###", "Task:", "Rules:"]  # More stop tokens
)

# --- System Prompt ---
SYSTEM_PROMPT = """Task: Determine if code needs migration from Python to C++.
Rules:
- If the code is Python: answer "Yes"
- If the code is C++ or other language: answer "No"
- Answer with only one word: "Yes" or "No"

Examples:
Code: def func(): pass
Answer: Yes

Code: #include <iostream>
Answer: No
"""

# --- vLLM Classification Function ---
def classify_vllm(snippets):
    prompts = [
        f"""{SYSTEM_PROMPT}

Code: {code}
Answer:"""
        for code in snippets
    ]

    outputs = llm_vllm.generate(prompts, sampling)

    def extract_answer(output):
        response = output.outputs[0].text.strip()
        print(f"vLLM raw output: '{response}'")  # Debug print

        # Clean up the response
        first_word = response.split()[0] if response.split() else ""

        if "yes" in first_word.lower():
            return "Yes"
        elif "no" in first_word.lower():
            return "No"
        return f"Unclear: {first_word}"

    return [extract_answer(o) for o in outputs]

# --- Test Snippets ---
python_snippet = """def add(a, b):
    return a + b
if __name__ == "__main__":
    print(add(2, 3))"""

cpp_snippet = """#include <iostream>
int add(int a, int b) {
    return a + b;
}
int main() {
    std::cout << add(2, 3) << std::endl;
    return 0;
}"""

javascript_snippet = """function add(a, b) {
    return a + b;
}
console.log(add(2, 3));"""

# --- Test vLLM Classification ---
print("=== Testing vLLM Classification ===")
test_snippets = [python_snippet, cpp_snippet, javascript_snippet]
labels = classify_vllm(test_snippets)

for i, (code, label) in enumerate(zip(test_snippets, labels), 1):
    print(f"\n--- Snippet {i} ---")
    lang = "Python" if "def " in code else "C++" if "#include" in code else "JavaScript"
    print(f"Language: {lang}")
    print(f"First line: {code.split(chr(10))[0]}")
    print(f"→ Needs migration to C++: {label}")

print("\n=== vLLM Classification Ready for Benchmarking ===")

In [ ]:
!pip install -q pandas

In [ ]:
%env TRANSFORMERS_VERBOSITY=error


In [ ]:
# --- Benchmark utilities ---
import time, math
import torch
import pandas as pd
import matplotlib.pyplot as plt

def gpu_sync():
    try:
        if torch.cuda.is_available():
            torch.cuda.synchronize()
    except Exception:
        pass

# Your three base snippets (already defined above, repeated here for clarity).
python_snippet = """def add(a, b):
    return a + b
if __name__ == "__main__":
    print(add(2, 3))"""

cpp_snippet = """#include <iostream>
int add(int a, int b) {
    return a + b;
}
int main() {
    std::cout << add(2, 3) << std::endl;
    return 0;
}"""

javascript_snippet = """function add(a, b) {
    return a + b;
}
console.log(add(2, 3));"""

BASE_SNIPPETS = [python_snippet, cpp_snippet, javascript_snippet]

def build_prompts(snippets, system_prompt: str, n: int):
    """Replicate the 3 base snippets to reach total size n."""
    unit = [
        f"""{system_prompt}

Code: {code}
Answer:"""
        for code in snippets
    ]
    reps = math.ceil(n / len(unit))
    prompts = (unit * reps)[:n]
    return prompts

def benchmark_hf(prompts, pipe, tokenizer, batch_size: int = 4, warmup: bool = True):
    """Return (elapsed_s, samples_per_s, tokens_per_s)."""
    # Warm-up (compile kernels / fill caches)
    if warmup and len(prompts) > 0:
        _ = pipe(prompts[:min(2, len(prompts))], return_full_text=False, batch_size=batch_size, num_return_sequences=1)
        try:
            import torch
            if torch.cuda.is_available():
                torch.cuda.synchronize()
        except Exception:
            pass

    t0 = time.perf_counter()
    outs = pipe(prompts, return_full_text=False, batch_size=batch_size, num_return_sequences=1)
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.synchronize()
    except Exception:
        pass
    elapsed = time.perf_counter() - t0

    # outs is List[List[{"generated_text": str}]]
    gen_texts = []
    for item in outs:
        # single prompt ⇒ item is a list; grab the first generated sequence
        first = item[0] if isinstance(item, list) else item
        gen_texts.append(first["generated_text"])

    # Count generated tokens (using the same tokenizer)
    gen_tokens = 0
    for txt in gen_texts:
        gen_tokens += len(tokenizer(txt, add_special_tokens=False).input_ids)

    n = len(prompts)
    sps = n / elapsed if elapsed > 0 else float("inf")
    tps = gen_tokens / elapsed if elapsed > 0 else float("inf")
    return elapsed, sps, tps


def benchmark_vllm(prompts, llm, sampling, warmup: bool = True):
    """Return (elapsed_s, samples_per_s, tokens_per_s)."""
    if warmup and len(prompts) > 0:
        _ = llm.generate(prompts[:min(2, len(prompts))], sampling)
        try:
            import torch
            if torch.cuda.is_available():
                torch.cuda.synchronize()
        except Exception:
            pass

    t0 = time.perf_counter()
    outs = llm.generate(prompts, sampling)
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.synchronize()
    except Exception:
        pass
    elapsed = time.perf_counter() - t0

    # Count tokens via the same HF tokenizer for parity with HF pipeline
    gen_tokens = 0
    for o in outs:
        if o.outputs:
            txt = o.outputs[0].text
            gen_tokens += len(tokenizer_hf(txt, add_special_tokens=False).input_ids)

    n = len(prompts)
    sps = n / elapsed if elapsed > 0 else float("inf")
    tps = gen_tokens / elapsed if elapsed > 0 else float("inf")
    return elapsed, sps, tps


# --- Run the benchmark ---
dataset_sizes = [3, 30, 300]
rows = []

# HF pipeline batch size (you initialized pipe_hf with batch_size=4; you can override per-call)
HF_BATCH_SIZE = 4

for n in dataset_sizes:
    prompts = build_prompts(BASE_SNIPPETS, SYSTEM_PROMPT, n)

    # HF
    hf_elapsed, hf_sps, hf_tps = benchmark_hf(prompts, pipe_hf, tokenizer_hf, batch_size=HF_BATCH_SIZE)
    rows.append({
        "engine": "HF/transformers",
        "dataset_size": n,
        "elapsed_s": hf_elapsed,
        "samples_per_s": hf_sps,
        "tokens_per_s": hf_tps
    })

    # vLLM
    v_elapsed, v_sps, v_tps = benchmark_vllm(prompts, llm_vllm, sampling)
    rows.append({
        "engine": "vLLM",
        "dataset_size": n,
        "elapsed_s": v_elapsed,
        "samples_per_s": v_sps,
        "tokens_per_s": v_tps
    })

# Tabulate
df = pd.DataFrame(rows).sort_values(["dataset_size", "engine"]).reset_index(drop=True)
print(df)

# Save CSV
df.to_csv("inference_benchmark.csv", index=False)

# --- Plot: dataset size vs samples/sec ---
plt.figure()
for eng in df["engine"].unique():
    sub = df[df["engine"] == eng]
    plt.plot(sub["dataset_size"], sub["samples_per_s"], marker="o", label=eng)

plt.xlabel("Dataset size (number of prompts)")
plt.ylabel("Throughput (samples/sec)")
plt.title("Inference Throughput vs Dataset Size (HF vs vLLM)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("inference_benchmark.png", dpi=200)
plt.show()

print("\nSaved:")
print(" - inference_benchmark.csv")
print(" - inference_benchmark.png")
